# User-Defined Functions (UDF)

While openEO supports a wide range of pre-defined processes and allows to build more complex user-defined processes from them, you sometimes need operations or algorithms that are not (yet) available or standardized as openEO process. User-Defined Functions (UDF) is an openEO feature (through the run_udf process) that aims to fill that gap by allowing a user to express (a part of) an algorithm as a Python/R/… script to be run back-end side.

Though several types of algorithms can be used as UDF applications, in this notebook, we showcase a simple example of how to work with UDF using the openEO Python Client library. 

In [2]:
# estabish connection to the backend and authenticate it
import openeo

connection = openeo.connect(url = "openeo-staging.creo.vito.be").authenticate_oidc()

Authenticated using refresh token.


In [7]:
# load collection

cube = connection.load_collection(
                            "SENTINEL2_L2A",
                            bands = ["B04", "B03", "B02"],
                            temporal_extent = ("2022-05-01", "2022-05-30"),
                            spatial_extent = {"west": 5.05, "south": 51.21, "east": 5.1, "north": 51.23,'crs': 'EPSG:4326'}

)

cube = cube.reduce_dimension(dimension = "t", reducer = "max")

Here the UDF code shown in the following cell does the actual value rescaling. 

In [8]:
# Build a UDF object from an inline string with Python source code.
udf = openeo.UDF("""
from openeo.udf import XarrayDataCube

def apply_datacube(cube: XarrayDataCube, context: dict) -> XarrayDataCube:
    array = cube.get_array()
    array.values = 0.0001 * array.values
    return cube
""")

User can also load their UDF from a seperate file using `openeo.UDF.from_file('my_udf.py')` and apply it along a dimension. 

In [9]:
# Apply the UDF to a cube.
rescaled_cube = cube.apply(process=udf)

In [10]:
rescaled_cube.download("rescale_s2.tiff")